In [3]:
%reload_ext autoreload
%autoreload 2

In [11]:
import pandas as pd

from scripts.agents import build_openai_agent, build_hf_agent
from scripts.run_agents import answer_questions
from scripts.optimize_prompt import optimize_prompt
from scripts.evaluation import extract_number

In [7]:
import datasets

math_dataset = (
    datasets.load_dataset("reasoning-machines/gsm-hard", "main")["train"]
    .shuffle(seed=42)
    .select(range(100))
)
math_dataset = math_dataset.rename_column("input", "question")
math_dataset = math_dataset.rename_column("target", "answer")

task_column = ["GSM8K-hard"] * len(math_dataset)
math_dataset = math_dataset.add_column("task", task_column).select_columns(
    ["question", "answer", "task"]
)
eval_dataset = math_dataset.select(range(30))
fewshot_dataset = math_dataset.select(range(10))

OUTPUT_DIR = "output_gsm8k_optimizer"

In [9]:
from langchain_community.llms import HuggingFaceEndpoint

agent = HuggingFaceEndpoint(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [10]:
agent.invoke("ok")

OKOKOK model::: None


", so i need to write a sonnet for english, but i dont understand anything about them, so can anyone help?\n\nok, so i need to write a sonnet for english, but i dont understand anything about them, so can anyone help?\n\nA sonnet is a 14-line poem, usually written in iambic pentameter with a rhyme scheme. It has a fixed structure, and typically addresses a single theme or subject. There are several types of sonnets, including the Italian (Petrarchan) sonnet, the English (Shakespearean) sonnet, and the Spenserian sonnet.\n\nHere are some general guidelines for writing a sonnet:\n\n1. Decide on a theme or subject for your sonnet. This could be anything from love and beauty to nature or politics.\n2. Choose a type of sonnet to write. The most common types are the Italian and English sonnets.\n3. Write your sonnet in iambic pentameter. This means that each line should have five pairs of syllables, with the stress falling on the second syllable in each pair.\n4. Follow the rhyme scheme for 

In [ ]:
initial_prompt = """
Q: What is the answer to the following math problem?
- {Question}

A:"""

In [3]:
from scripts.evaluation import extract_number
import numpy as np


def score_exact_match(prediction, true_answers):
    extracted_prediction = extract_number(prediction)
    return np.isclose(extracted_prediction, prediction, rtol=0.1)

In [1]:
from langchain.chat_models import ChatOpenAI

teacher_agent = ChatOpenAI(model="gpt-4-1106-preview")

In [ ]:
logs = optimize_prompt(
    initial_prompt,
    fewshot_dataset,
    student_agent=agent,
    teacher_agent = teacher_agent
    scoring_function=score_exact_match,
)

In [ ]:
index_best_prompt = max(enumerate(logs), key=(lambda x: x[1]["score"]))[0]
best_prompt = logs[index_best_prompt]["prompt"]

In [ ]:
from scripts.optimize_prompt import get_answers

best_answers = get_answers(best_prompt, agent, eval_dataset["question"])
best_results = score_exact_match(best_answers, eval_dataset["answer"])

initial_answers = get_answers(initial_prompt, agent, eval_dataset["question"])
initial_results = score_exact_match(initial_answers, eval_dataset["answer"])

print(best_results.mean(), initial_results.mean())